In [78]:
import array, csv, json, math
from osgeo import ogr
from random import shuffle, uniform
from shapely.geometry import shape, Point

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def RandomPointFromPolygon(geom):
    polygon = shape(geom)
    bbox = polygon.bounds
    l,b,r,t = bbox
    while True:
        point = Point(uniform(l,r),uniform(t,b))
        if point is None:
            break
        if polygon.contains(point):
            break
    return point.__geo_interface__['coordinates']

In [ ]:
# Transform NI Small Areas shapefile into GeoJSON
cmd = "ogr2ogr -f GeoJSON -t_srs crs:84 SA2011/SA2011.geojson SA2011/SA2011.shp"
!$cmd

In [7]:
# Load small areas GeoJSON file
with open("SA2011/SA2011.geojson") as f:
    sa2011 = json.load(f)

In [8]:
#sa2011['features'][0]['properties']
#{u'Hectares': 169.044,
# u'SA2011': u'N00000002',
# u'SOA2011': u'95AA01S2',
# u'X_COORD': 315465.0,
# u'Y_COORD': 377176.0}

In [9]:
belfastData = []
with open("Geographic Data (statistical geographies).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['HSCT'] == "Belfast HSCT":
            belfastData.append(row)

In [10]:
#belfastData[0]
#{'AA1998': 'BELFAST WEST',
# 'AA2008': 'BELFAST WEST',
# 'HSCT': 'Belfast HSCT',
# 'LGD1992NAME': 'BELFAST',
# 'SA': 'N00000825',
# 'SOA': '95GG01S1',
# 'SQ Km': '0.11',
# 'Settlement2015(1)': 'BELFAST CITY',
# 'Settlement2015(2)': '',
# 'Settlement2015(3)': '',
# 'URBAN_RURAL(2015)': 'Urban',
# 'WARD1992': 'ANDERSONSTOWN'}

In [11]:
belfastGeoJSON = { 
    "type": "FeatureCollection",
    "features": []
}
for row in belfastData:
    sa = row['SA']
    for feature in sa2011['features']:
        if feature['properties']['SA2011'] == sa:
            for key in row.keys():
                feature['properties'][key] = row[key]
            belfastGeoJSON['features'].append(feature)
            break
with open('belfast.geojson', 'w') as outfile:  
    json.dump(belfastGeoJSON, outfile, indent=4)

In [12]:
belfastSa2Idx = {}
for i in range(0,len(belfastGeoJSON["features"])):
    feature = belfastGeoJSON["features"][i]
    sa = feature['properties']['SA']
    belfastSa2Idx[sa] = i
len(belfastSa2Idx)

1008

In [13]:
religionData = []
with open("_KS211NI (s).ods - SA.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['SA Code'] in belfastSa2Idx:
            religionData.append(row)
len(religionData)

1008

In [14]:
#religionData[0]
#{'All usual residents': '367',
# 'Religion: Catholic': '320',
# 'Religion: Catholic (%)': '87.19',
# 'Religion: Church of Ireland': '0',
# 'Religion: Church of Ireland (%)': '0.00',
# 'Religion: Methodist Church in Ireland': '0',
# 'Religion: Methodist Church in Ireland (%)': '0.00',
# 'Religion: No religion': '15',
# 'Religion: No religion (%)': '4.09',
# 'Religion: Other Christian (including Christian related)': '3',
# 'Religion: Other Christian (including Christian related) (%)': '0.82',
# 'Religion: Other religions': '0',
# 'Religion: Other religions (%)': '0.00',
# 'Religion: Presbyterian Church in Ireland': '2',
# 'Religion: Presbyterian Church in Ireland (%)': '0.54',
# 'Religion: Religion not stated': '27',
# 'Religion: Religion not stated (%)': '7.36',
# 'SA': 'N00000825 (Andersonstown ward)',
# 'SA Code': 'N00000825'}

In [47]:
romanCatholic = ['Religion: Catholic']
presbyterian = ['Religion: Presbyterian Church in Ireland']
churchOfIreland = ['Religion: Church of Ireland']
methodist = ['Religion: Methodist Church in Ireland']
other = ['Religion: No religion', 'Religion: Other Christian (including Christian related)', 
         'Religion: Other religions', 'Religion: Religion not stated']

religions = {
    'romanCatholic':romanCatholic, 
    'presbyterian': presbyterian, 
    'churchOfIreland': churchOfIreland, 
    'methodist': methodist, 
    'other': other}

religion2Idx = {
    'romanCatholic': 1.0, 
    'presbyterian': 2.0, 
    'churchOfIreland': 3.0, 
    'methodist': 4.0, 
    'other': 5.0
}

In [32]:
geom = belfastGeoJSON["features"][0]["geometry"]
p = RandomPointFromPolygon(geom)
LonLatToPixelXY(p)

[123.73697328436603, 81.50378179192757]

In [62]:
points_2011 = []
for row in religionData:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = int(row['All usual residents'].replace(',', ''))
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += int(row[i].replace(',',''))
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_2011.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
        print mis        

In [63]:
len(points_2011)

348204

In [72]:
shuffle(points_2011)

In [74]:
points = []
for sublist in points_2011:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-2011.bin', 'wb'))

In [79]:
lookupTableData = []
with open("Look-up Tables_0.xlsx - SA2011.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['SA2011'] in belfastSa2Idx:
            lookupTableData.append(row)
len(lookupTableData)    

1008

In [81]:
#lookupTableData[0]
#{'COA2001_1': '95GG010001',
#'COA2001_2': '',
# 'COA2001_3': '',
# 'COA2001_4': '',
# 'SA2011': 'N00000825',
# 'SA2011NAME': 'N00000825 (Andersonstown ward)',
# 'SOA2001': '95GG01S1',
# 'SOANAME': 'Andersonstown_1'}

In [85]:
raw_2001 = []
with open("2001/oa/TABLE CAS308 (OA).csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        raw_2001.append(row)


In [121]:
religionDataIdx_2001 = {}

for row in lookupTableData:
    idx_2001 = row['COA2001_1']
    religionDataIdx_2001[idx_2001] = []
    for raw_row in raw_2001:
        if len(raw_row) == 3:
            raw_idx = raw_row[0].strip()
            raw_key = raw_row[1]
            raw_value = raw_row[2]
            if raw_value == "-":
                raw_value = '0'
            if raw_idx == idx_2001:
                religionDataIdx_2001[idx_2001].append({raw_key: raw_value})
            

In [124]:
religionData_2001 = []
for row in lookupTableData:
    data_2001 = {}
    idx_2001 = row['COA2001_1']
    data_2001['SA Code'] = row['SA2011']
    data_2001['COA2001_1'] = idx_2001
    data_2001['All usual residents'] = religionDataIdx_2001[idx_2001][0]['All persons']
    data_2001['Religion: Catholic'] = religionDataIdx_2001[idx_2001][2]['Catholic']
    data_2001['Religion: Church of Ireland'] = religionDataIdx_2001[idx_2001][4]['Church of Ireland']
    data_2001['Religion: Methodist Church in Ireland'] = religionDataIdx_2001[idx_2001][5]['Methodist Church in Ireland']
    data_2001['Religion: Presbyterian Church in Ireland'] = religionDataIdx_2001[idx_2001][3]['Presbyterian Church in Ireland']
    data_2001['Religion: Other Christian (including Christian related)'] = religionDataIdx_2001[idx_2001][6]['Other Christian (including Christian related)']
    data_2001['Religion: Other religions'] = religionDataIdx_2001[idx_2001][7]['Other religions and philosophies']
    data_2001['Religion: No religion'] = religionDataIdx_2001[idx_2001][8]['No religion or religion not stated']
    data_2001['Religion: Religion not stated'] = '0'
    religionData_2001.append(data_2001)
    
    

In [125]:
religionData_2001[0]

{'All usual residents': '432',
 'COA2001_1': '95GG010001',
 'Religion: Catholic': '411',
 'Religion: Church of Ireland': '3',
 'Religion: Methodist Church in Ireland': '0',
 'Religion: No religion': '18',
 'Religion: Other Christian (including Christian related)': '0',
 'Religion: Other religions': '0',
 'Religion: Presbyterian Church in Ireland': '0',
 'Religion: Religion not stated': '0',
 'SA Code': 'N00000825'}

In [117]:
religionDataIdx_2001['95GG010002']

[{'All persons': '415'},
 {'': '*'},
 {'Catholic': '393'},
 {'Presbyterian Church in Ireland': 0},
 {'Church of Ireland': 0},
 {'Methodist Church in Ireland': 0},
 {'Other Christian (including Christian related)': 0},
 {'Other religions and philosophies': 0},
 {'No religion or religion not stated': '22'}]

In [115]:
data_2001

{'COA2001_1': '95GG010001', 'SA Code': 'N00000825'}

In [126]:
religionDataIdx_2001[idx_2001]

[{'All persons': '294'},
 {'': '*'},
 {'Catholic': '130'},
 {'Presbyterian Church in Ireland': '43'},
 {'Church of Ireland': '27'},
 {'Methodist Church in Ireland': '26'},
 {'Other Christian (including Christian related)': '13'},
 {'Other religions and philosophies': '0'},
 {'No religion or religion not stated': '55'}]

In [127]:
points_2001 = []
for row in religionData_2001:
    sa = row['SA Code']
    featureIdx = belfastSa2Idx[sa]
    geom = belfastGeoJSON["features"][featureIdx]['geometry']
    total = int(row['All usual residents'].replace(',', ''))
    totals = 0
    for k in religions.keys():
        currentTotal = 0
        for i in religions[k]:
            currentTotal += int(row[i].replace(',',''))
        for i in range(currentTotal):
            point = []
            point += LonLatToPixelXY(RandomPointFromPolygon(geom))
            point.append(religion2Idx[k])
            points_2001.append(point)
        totals += currentTotal        
    if total != totals:
        mis += 1
        print mis        

In [128]:
len(points_2001)

309518

In [129]:
shuffle(points_2001)

In [130]:
points = []
for sublist in points_2001:
    for item in sublist:
        points.append(item)
array.array('f', points).tofile(open('dotmap-2001.bin', 'wb'))